# Create a Simulated LFT Operations SQLITE DB

From wikipedia:

*Lateral flow tests (LFTs), also known as lateral flow immunochromatographic assays or rapid tests,
are simple devices intended to detect the presence of a target substance in a liquid sample without
the need for specialized and costly equipment. These tests are widely used in medical diagnostics
for home testing, point of care testing, or laboratory use.
As part of the response to the COVID-19 pandemic, the National Health Service is issuing LFT for
home-use. The device is simple to use, cheap to produce, and results are available in approximately
30 minutes. If a positive result is encountered, follow up PCR testing is to be performed.
Many lab-based applications increase the sensitivity of simple LFTs by employing additional dedicated equipment.
It is on this principle that the simulated quantiative dataset (as opposed to the commonly used
qualitative LFT such as that issued by the NHS) for use as part of this demo project is being
prepared.*

Quantitative assays yield more interesting datasets.

Dummy data for each of the key components of a simplifed LFT device are individually simulated,
as are the roles of a simplified operations team. Each component will contribute tothe precision,
bias and reliability of the device.


In [1]:
!rm operations.db

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

sql_db = create_engine('sqlite:///operations.db')
materials_list = pd.read_csv('materials_list.csv', index_col = False).loc[0:8,] # Only RMs

In [3]:
materials_list

,Name,field_name,part_no,bias_min,bias_max,prec_min,prec_max,reli_min,reli_max
0,Conjugate Pad,conj_pad,E-OPS-0001,0.9,1.1,3.3,5.0,0.01,0.04
1,Sample Pad,smpl_pad,E-OPS-0002,0.9,1.1,3.3,5.0,0.01,0.04
2,Wicking Pad,wick_pad,E-OPS-0003,0.9,1.1,3.3,5.0,0.01,0.04
3,Detection Latex,det_latex,E-OPS-0004,0.9,1.1,3.3,5.0,0.01,0.04
4,Detection Antibody,det_ab,E-OPS-0005,0.9,1.1,3.3,5.0,0.01,0.04
5,Test Antibody,test_ab,E-OPS-0006,0.9,1.1,3.3,5.0,0.01,0.04
6,Control Antibody,ctrl_ab,E-OPS-0007,0.9,1.1,3.3,5.0,0.01,0.04
7,Antibody Diluent,diluent,E-OPS-0008,0.9,1.1,3.3,5.0,0.01,0.04
8,Plastic Cassette,cassette,E-OPS-0009,0.9,1.1,3.3,5.0,0.01,0.04


In [4]:
def new_mfr_lot(alpha_id = 3, numeric_id = 7):
    alpha_part = np.random.randint(65,90,alpha_id)
    alpha_part = [chr(alpha_part[i]) for i in range(0, len(alpha_part))]
    numeric_part = str(np.random.randint(0,10**numeric_id))
    
    return ''.join(alpha_part) + numeric_part

def first_prep(mat_list = materials_list):
    build_list = []
    for index, row in mat_list.iterrows():
        name = row['Name']
        lot_no = new_mfr_lot()
        bias = np.random.uniform(row['bias_min'], row['bias_max'], 1)[0]
        prec = np.random.uniform(row['prec_min'], row['prec_max'], 1)[0]
        reli = np.random.uniform(row['prec_min'], row['prec_max'], 1)[0]
        build_list.append([name, lot_no, bias, prec, reli])
        
    return pd.DataFrame(build_list, columns = ['name', 'lot_no', 'bias', 'prec', 'reli'])

def new_prep(old_build_list, mat_list = materials_list):
    new_build_list = []
    for index, row in old_build_list.iterrows():
        if np.random.uniform() < 0.9:
            new_build_list.append(row.values)
        else:
            name = row['name']
            lot_no = new_mfr_lot()
            temp = mat_list.query('Name == @name')
            bias = np.random.uniform(temp['bias_min'], temp['bias_max'], 1)[0]
            prec = np.random.uniform(temp['prec_min'], temp['prec_max'], 1)[0]
            reli = np.random.uniform(temp['prec_min'], temp['prec_max'], 1)[0]
            new_build_list.append([name, lot_no, bias, prec, reli])
    
    return pd.DataFrame(new_build_list, columns = ['name', 'lot_no', 'bias', 'prec', 'reli'])

In [5]:
prep = first_prep()

for lot in range(1, 5000):
    if np.random.binomial(1,0.05):
        pass
    else:
        # Set Lot and RMs
        conv_lot = 302100 + lot
        prep = new_prep(prep)
        
        # Raw Material DB Entry
        temp = pd.DataFrame(columns = ['lot'] + list(materials_list.field_name))
        temp.loc[len(temp)] = [conv_lot] + list(prep.lot_no)
        temp.to_sql(name = 'raw_materials', con = sql_db, if_exists = 'append', index = False)
        
        # Calculate Unknown Lot Performance Measures
        bias = sum(prep.bias - 1)
        prec = sum(prep.prec ** 2)**0.5
        reli = sum(prep.reli)
        
        # QC Performance Testing
        
        low_test = np.random.normal(loc = 1000 * (1 + bias), scale = 1000 * (1 + bias) * (prec / 100), size = 100)
        low_errors = np.random.binomial(p = reli / 1000, n = 1, size = 100)
        high_test = np.random.normal(loc = 5000 * (1 + bias), scale = 5000 * (1 + bias) * (prec / 100), size = 100)
        high_errors = np.random.binomial(p = reli / 1000, n = 1, size = 100)
        low_test[low_errors] = np.nan
        high_test[high_errors] = np.nan

        low_errors = pd.Series(low_errors * np.random.randint(10,20,100)).astype(str)
        high_errors = pd.Series(high_errors * np.random.randint(10,20,100)).astype(str)

        qc_testing = pd.DataFrame({"lot": [conv_lot for x in range(0,200)], 
                                   "level": [1 for x in range(0,100)] + [2 for x in range(0,100)], 
                                   "result": list(low_test) + list(high_test),  
                                   "error_code": list(low_errors) + list(high_errors)})

        qc_testing.to_sql(name = 'qc_test', con = sql_db, if_exists = 'append', index = False)
